# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
I. Momcheva  ->  I. Momcheva  |  ['I. Momcheva']
J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
F. Walter  ->  F. Walter  |  ['F. Walter']
J. Liu  ->  J. Liu  |  ['J. Liu']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Arxiv has 75 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.13945


extracting tarball to tmp_2506.13945...

 done.


Found 82 bibliographic references in tmp_2506.13945/mnras_euclidqso.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.14366


...

/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/Usco_map.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width =0.5\textwidth]{Figures/Usco_map.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2506.14519


extracting tarball to tmp_2506.14519...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2506.14519/v3.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'main_sub/tab_pol' from 'tmp_2506.14519/main_sub/tab_pol.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'main_sub/fig4pol' from 'tmp_2506.14519/main_sub/fig4pol.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.13945-b31b1b.svg)](https://arxiv.org/abs/2506.13945) | **Euclid: The potential of slitless infrared spectroscopy: A z=5.4 quasar and new ultracool dwarfs**  |
|| <mark>E. Bañados</mark>, et al. -- incl., <mark>S. Belladitta</mark>, <mark>I. Momcheva</mark>, <mark>J. Wolf</mark>, <mark>H.-W. Rix</mark>, <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-06-18*|
|*Comments*| *Euclid NISP RGE quasar spectrum in Fig. 5 and NISP BGE spectrum in Appendix Fig. C1. Submitted*|
|**Abstract**|            We demonstrate the potential of Euclid's slitless spectroscopy to discover high-redshift (z>5) quasars and their main photometric contaminant, ultracool dwarfs. Sensitive infrared spectroscopy from space is able to efficiently identify both populations, as demonstrated by Euclid Near-Infrared Spectrometer and Photometer Red Grism (NISP RGE) spectra of the newly discovered z=5.404 quasar EUCL J181530.01+652054.0, as well as several ultracool dwarfs in the Euclid Deep Field North and the Euclid Early Release Observation field Abell 2764. The ultracool dwarfs were identified by cross-correlating their spectra with templates. The quasar was identified by its strong and broad CIII] and MgII emission lines in the NISP RGE 1206-1892 nm spectrum, and confirmed through optical spectroscopy from the Large Binocular Telescope. The NISP Blue Grism (NISP BGE) 926-1366 nm spectrum confirms CIV and CIII] emission. NISP RGE can find bright quasars at z~5.5 and z>7, redshift ranges that are challenging for photometric selection due to contamination from ultracool dwarfs. EUCL J181530.01+652054.0 is a high-excitation, broad absorption line quasar detected at 144 MHz by the LOw-Frequency Array (L144=4e25 W/Hz). The quasar has a bolometric luminosity of 3e12 Lsun and is powered by a 3.4e9 Msun black hole. The discovery of this bright quasar is noteworthy as fewer than one such object was expected in the ~20 deg2 surveyed. This finding highlights the potential and effectiveness of NISP spectroscopy in identifying rare, luminous high-redshift quasars, previewing the census of these sources that Euclid's slitless spectroscopy will deliver over about 14,000 deg2 of the sky.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.14380-b31b1b.svg)](https://arxiv.org/abs/2506.14380) | **Multiplicity of young isolated planetary mass objects in Taurus and Upper Scorpius**  |
|| H. Bouy, et al. -- incl., <mark>W. Brandner</mark> |
|*Appeared on*| *2025-06-18*|
|*Comments*| *Submitted to A&A on 2025-04-04 ; awaiting the initial review*|
|**Abstract**|            Free-floating planetary mass objects-worlds that roam interstellar space untethered to a parent star-challenge conventional notions of planetary formation and migration, but also of star and brown dwarf formation. We focus on the multiplicity among free-floating planets. By virtue of their low binding energy (compared to other objects formed in these environments), these low-mass substellar binaries represent a most sensitive probe of the mechanisms at play during the star formation process. We use the Hubble Space Telescope and its Wide Field Camera 3 and the Very Large Telescope and its ERIS adaptive optics facility to search for visual companions among a sample of 77 objects members of the Upper Scorpius and Taurus young nearby associations with estimated masses in the range between approximately 5-33 M$_{\rm Jup}$. We report the discovery of one companion candidate around a Taurus member with a separation of 111.9$\pm$0.4 mas, or $\sim$18 au assuming a distance of 160pc, with an estimated primary mass in the range between 3-6 M$_{\rm Jup}$ and a secondary mass between 2.6-5.2 M$_{\rm Jup}$, depending on the assumed age. This corresponds to an overall binary fraction of 1.8$^{+2.6}_{-1.3}$% among free-floating planetary mass objects over the separation range $\ge$7 au. Despite the limitations of small-number statistics and variations in spatial resolution and sensitivity, our results, combined with previous high-spatial-resolution surveys, suggest a notable difference in the multiplicity properties of objects below $\sim$25 M$_{\rm Jup}$ between Upper Sco and Taurus. In Taurus, five companions were identified among 78 observed objects (4.9$^{+2.8}_{-2.0}$%), whereas none were found among 97 objects in Upper Sco ($\le$1.2%).}         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.14366-b31b1b.svg)](https://arxiv.org/abs/2506.14366) | **The power of SKA to Constrain cosmological gravitational-wave backgrounds below the astrophysical foreground noise**  |
|| C. Fu, <mark>J. Liu</mark> |
|*Appeared on*| *2025-06-18*|
|*Comments*| *8 pages, 2 figures*|
|**Abstract**|            The inspirals of supermaissive black hole binaries provide a convinced gravitational wave background in the nHz band, serving as the fiducial model of the recent gravitational wave signal reported by the PTA experiments. The uncertainties of the number of binaries contributing to each frequency bin introduce a foreground noise in the nHz and $\mu$Hz bands against the observation of the underlying gravitational wave backgrounds of the cosmological origin. In this work, we investigate a new method to constrain the cosmological gravitational wave strength under the astrophysical foreground. The energy density fluctuations from cosmological gravitational-wave sources can generally trigger the formation of compact subhalos of dark matter, and the upcoming Square Kilometer Array has the ability to constrain the abundance of the subhalos at the $\mathcal{O}(1)$ level. The cosmological gravitational wave energy spectra from various sources are expected to be constrained several orders of magnitude below the astrophysical foreground, providing more strict constraints on the parameter spaces of corresponding new physics models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.14519-b31b1b.svg)](https://arxiv.org/abs/2506.14519) | **FAST Pulsar Database: II. Scattering profiles of 122 Pulsars**  |
|| W. C. Jing, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-06-18*|
|*Comments*| *Subband profiles of each pulsar are shown in the paper. Accepted by RAA*|
|**Abstract**|            The turbulent ionized interstellar medium diffracts radio waves and makes them propagate in multiple paths. The pulse-broadening observed at low frequencies results from the scattering effect of interstellar clouds of ionized gas. During the Galactic Plane Pulsar Snapshot (GPPS) survey and other projects by using the Five-hundred-meter Aperture Spherical radio Telescope (FAST), we detect the pulse-broadening for 122 pulsars in the radio frequency band between 1.0 and 1.5 GHz, including 60 newly discovered pulsars in the GPPS survey and 62 previously known pulsars. We find that a more accurate dispersion measure can be obtained from aligning the front edge of the scattered subband pulses at the 1/4 or 1/2 peak level for most pulsars with one dominant component in the intrinsic profile, and the best DM values from aligning the intrinsic profile components from the model-fitting. From the pulse profiles at a few subbands we derive the pulse-broadening timescale and the scattering spectral index. These scattering parameters are measured for the first time for 93 pulsars. For 29 pulsars with previously detected scattering features, our measurements of the pulse-broadening timescale are consistent with results in the literature. We find that pulsars behind spiral arms show a stronger scattering effect due to greater density fluctuations in the arm regions. With a properly derived dispersion measure and careful calibration, we also present polarization profiles for 41 pulsars in three subbands of FAST observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.13945/./f5.png', 'tmp_2506.13945/./f1.png', 'tmp_2506.13945/./f3.png']
copying  tmp_2506.13945/./f5.png to _build/html/
copying  tmp_2506.13945/./f1.png to _build/html/
copying  tmp_2506.13945/./f3.png to _build/html/
exported in  _build/html/2506.13945.md
    + _build/html/tmp_2506.13945/./f5.png
    + _build/html/tmp_2506.13945/./f1.png
    + _build/html/tmp_2506.13945/./f3.png
found figures ['tmp_2506.14380/./Figures/galaxy.png', 'tmp_2506.14380/./Figures/targets_i_iz.png', 'tmp_2506.14380/./Figures/contrast_hst.png']
copying  tmp_2506.14380/./Figures/galaxy.png to _build/html/
copying  tmp_2506.14380/./Figures/targets_i_iz.png to _build/html/
copying  tmp_2506.14380/./Figures/contrast_hst.png to _build/html/
exported in  _build/html/2506.14380.md
    + _build/html/tmp_2506.14380/./Figures/galaxy.png
    + _build/html/tmp_2506.14380/./Figures/targets_i_iz.png
    + _build/html/tmp_2506.14380/./Figures/contrast_hst.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sfont}[1]{{\scriptscriptstyle\rm #1}}$
$\newcommand{\IE}{\ensuremath{I_\sfont{E}}\xspace}$
$\newcommand{\YE}{\ensuremath{Y_\sfont{E}}\xspace}$
$\newcommand{\JE}{\ensuremath{J_\sfont{E}}\xspace}$
$\newcommand{\HE}{\ensuremath{H_\sfont{E}}\xspace}$
$\newcommand{\BGE}{\ensuremath{{\rm BG}_\sfont{E}}\xspace}$
$\newcommand{\RGE}{\ensuremath{{\rm RG}_\sfont{E}}\xspace}$
$\newcommand{\quasar}{EUCL J181530.01+652054.0}$
$\newcommand{\quasarshort}{EUCL~QSO J1815+6520}$
$\newcommand{\noquasar}{EUCL J180409.14+641335.3}$
$\newcommand{\tdwarf}{WISE~J174556.65+645933.8}$
$\newcommand{\tdwarfeuclid}{EUCL J174556.41+645936.8}$
$\newcommand{\mdwarf}{EUCL J174429.80+672728.1}$
$\newcommand{\dwarfabell}{EUCL J002516.31-491618.5}$
$\newcommand{\ldwarf}{EUCL J181201.83+661609.6}$
$\newcommand{\AckEC}{The Euclid Consortium acknowledges the European$
$  Space Agency and \AckInstitutions}$
$\newcommand{\pd}{\phantom{1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand\aliii{\ensuremath{\text{Al \textsc{iii}}}}$
$\newcommand\siiv{\ensuremath{\text{Si \textsc{iv}}}}$
$\newcommand\civ{\ensuremath{\text{C \textsc{iv}}}}$
$\newcommand\mgii{\ensuremath{\text{Mg \textsc{ii}}}}$
$\newcommand\heii{\ensuremath{\text{He \textsc{ii}}}}$
$\newcommand\feii{\ensuremath{\text{Fe \textsc{ii}}}}$
$\newcommand\feiii{\ensuremath{\text{Fe \textsc{iii}}}}$
$\newcommand\cii{\ensuremath{\text{[C \textsc{ii]}}}}$
$\newcommand\ciii{\ensuremath{\text{C \textsc{iii]}}}}$
$\newcommand\ciilong{\ensuremath{\text{[C \textsc{ii]}}_{158 \mu\text{m}}}}$
$\newcommand\heii{\ensuremath{\text{He \textsc{ii}}}}$
$\newcommand\halpha{\ensuremath{\text{H}\alpha}}$
$\newcommand\hbeta{\ensuremath{\text{H}\beta}}$
$\newcommand\lya{\ensuremath{\text{Ly}\alpha}}$
$\newcommand\nv{\ensuremath{\text{N \textsc{v}}}}$</div>



<div id="title">

# $\Euclid$: The potential of slitless infrared spectroscopy: \\A $z=5.4$ quasar and new ultracool dwarfs$\thanks{This paper is published on behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.13945-b31b1b.svg)](https://arxiv.org/abs/2506.13945)<mark>Appeared on: 2025-06-18</mark> -  _Euclid NISP RGE quasar spectrum in Fig. 5 and NISP BGE spectrum in Appendix Fig. C1. Submitted_

</div>
<div id="authors">

<mark>E. Bañados</mark>, et al. -- incl., <mark>S. Belladitta</mark>, <mark>I. Momcheva</mark>, <mark>J. Wolf</mark>, <mark>H.-W. Rix</mark>, <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** $\footnotesize$ We demonstrate the potential of $\Euclid{\rm 's}$ slitless spectroscopy to discover high-redshift ( $z>5$ ) quasars and their main photometric contaminant, ultracool dwarfs. Sensitive infrared spectroscopy from space is able to efficiently identify both populations, as demonstrated by $\Euclid$ Near-Infrared Spectrometer and Photometer Red Grism (NISP $\RGE$ ) spectra of the newly discovered $z=5.404$ quasar $\quasar$ , as well as several ultracool dwarfs in the Euclid Deep Field North and the _Euclid_ Early Release Observation field Abell 2764.The ultracool dwarfs were identified by cross-correlating their spectra with templates. The quasar was identified by its strong and broad $\ciii$ and $\mgii$ emission lines in the NISP $\RGE$ 1206--1892 nm spectrum, and confirmed through optical spectroscopy from the Large Binocular Telescope. The NISP Blue Grism (NISP $\BGE$ ) 926--1366 nm spectrum confirms $\civ$ and $\ciii$ emission.NISP $\RGE$ can find bright quasars at $ z\approx 5.5$ and $ z\gtrsim 7$ , redshift ranges that are challenging for photometric selection due to contamination from ultracool dwarfs. $\quasar$ is a high-excitation, broad absorption line quasar detected at 144 MHz by the LOw-Frequency Array ( $L_{\rm 144}=4.0 \times 10^{25} $ W Hz $^{-1}$ ). The quasar has a bolometric luminosity of $3\times 10^{12} \lsun$ and is powered by a $3.4\times 10^9 \msun$ black hole. The discovery of this bright quasar is noteworthy as fewer than one such object was expected in the $\approx$ 20 deg $^2$ surveyed. This finding highlights the potential and effectiveness of NISP spectroscopy in identifying rare, luminous high-redshift quasars, previewing the census of these sources that $\Euclid$ 's slitless spectroscopy will deliver over about $14 000 $ deg $^2$ of the sky.

</div>

<div id="div_fig1">

<img src="tmp_2506.13945/./f5.png" alt="Fig9" width="100%"/>

**Figure 9. -** _Top:_ Postage stamps of the $z=5.4$ quasar $\quasar$. The \Euclid$\IE$, $\YE$, $\JE$, and $\HE$ images are $5$\arcsec$$ on a side while the LOFAR image is $30$\arcsec$$ on a side. The LOFAR   beam is shown on the lower left of its panel.
    _Middle:_  NISP grism $\RGE$ spectrum (black line; masked pixels are in grey, and the uncertainties in green). The blue line shows the best-fit power-law emission plus \ciii and \mgii broad emission lines, identifying this as a quasar at $z_{\rm \mgii}=5.404\pm 0.007$.
    _Bottom: _ LBT/MODS optical spectrum (black line and uncertainties in green), confirming the quasar nature of $\quasar$. The vertical dashed lines correspond to the expected position of the labelled emission lines based on the \mgii redshift.
    The dashed rectangles indicate the \siiv and \civ BAL regions shown in Fig. \ref{fig:bal}.
     (*fig:qso*)

</div>
<div id="div_fig2">

<img src="tmp_2506.13945/./f1.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
    Quasar strong emission lines as a function of redshift within the NISP **$\BGE$** and $\RGE$ bandpasses  ([Gillard and Schirmer 2025]()) , as indicated at the top of the figure. NISP $\BGE$ data will only be available in the EDS, while NISP $\RGE$ data will be present in the EWS.  The vertical grey-shaded regions correspond to wavelengths of strong telluric absorption, where ground-based telescopes are not sensitive. The horizontal red-shaded regions represent the redshift ranges with at least two strong emission lines expected in the NISP $\RGE$ spectral bandpass, thereby providing the most reliable redshifts for `blind' discoveries over the entire EWS.
   (*fig:motivation*)

</div>
<div id="div_fig3">

<img src="tmp_2506.13945/./f3.png" alt="Fig7" width="100%"/>

**Figure 7. -** _Top:_ Postage stamps of the T3 dwarf  $\dwarfabell$.  The \Euclid$\IE$, $\YE$, $\JE$, and $\HE$ images are $5$\arcsec$$ on a side.
    _Bottom:_ NISP $\RGE$ grism spectrum (black line and the uncertainties in green). The blue line shows a template of a T3 binary: 2MASS J12095613$-$1004008  ([Burgasser, et. al 2004](https://ui.adsabs.harvard.edu/abs/2004AJ....127.2856B), [Dupuy and Liu 2012](http://adsabs.harvard.edu/abs/2012ApJS..201...19D)) .
     (*fig:a2769*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.13945"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mjup}{M_{\rm Jup} }$
$\newcommand{\GD}[1]{ {\color{blue}(\it Gaspard: #1)}}$
$\newcommand{\GS}[1]{ {\color{teal}(\it Giovanni: #1)}}$
$\newcommand{\JA}[1]{ {\color{goldenrod}(\it Jonathan: #1)}}$
$\newcommand{\HB}[1]{ {\color{magenta}(\it Hervé: #1)}}$
$\newcommand{\straklip}{\textit{Stra}KLIP}$
$\newcommand{\publicwifi}{\textit{PUBLIC-WiFi}}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Multiplicity of young isolated planetary mass objects in Taurus and Upper Scorpius

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.14380-b31b1b.svg)](https://arxiv.org/abs/2506.14380)<mark>Appeared on: 2025-06-18</mark> -  _Submitted to A&A on 2025-04-04 ; awaiting the initial review_

</div>
<div id="authors">

H. Bouy, et al. -- incl., <mark>W. Brandner</mark>

</div>
<div id="abstract">

**Abstract:** Free-floating planetary mass objects--worlds that roam interstellar space untethered to a parent star--challenge conventional notions of planetary formation and migration, but also of star and brown dwarf formation. We focus on the multiplicity among free-floating planets. By virtue of their low binding energy (compared to other objects formed in these environments), these low-mass substellar binaries represent a most sensitive probe of the mechanisms at play during the star formation process. We use the $_ Hubble Space Telescope_$ and its Wide Field Camera 3 and the $_ Very Large Telescope_$ and its ERIS adaptive optics facility to search for visual companions among a sample of 77 objects members of the Upper Scorpius and Taurus young nearby associations with estimated masses in the range between approximately 5--33 $\mjup$ . We report the discovery of one companion candidate around a Taurus member with a separation of 111.9 $\pm$ 0.4 mas, or $\sim$ 18 au assuming a distance of 160 pc, with an estimated primary mass in the range between 3--6 $\mjup$ and a secondary mass between 2.6--5.2 $\mjup$ , depending on the assumed age. This corresponds to an overall binary fraction of 1.8 $^{+2.6}_{-1.3}$ \% among free-floating planetary mass objects over the separation range $\ge$ 7 au.  Despite the limitations of small-number statistics and variations in spatial resolution and sensitivity, our results, combined with previous high-spatial-resolution surveys, suggest a notable difference in the multiplicity properties of objects below $\sim$ 25 $\mjup$ between Upper Sco and Taurus. In Taurus, five companions were identified among 78 observed objects (4.9 $^{+2.8}_{-2.0}$ \% ), whereas none were found among 97 objects in Upper Sco ( $\le$ 1.2 \% ).

</div>

<div id="div_fig1">

<img src="tmp_2506.14380/./Figures/galaxy.png" alt="Fig6" width="100%"/>

**Figure 6. -** HST F814W (left) and F850LP (right) images of the visual companion identified around J164636.12-231337.6 through direct inspection. A square root stretch is used. Contours highlight the extended emission originating from the visual companion. North is up and east is left. (*fig:galaxy-comp*)

</div>
<div id="div_fig2">

<img src="tmp_2506.14380/./Figures/targets_i_iz.png" alt="Fig1" width="100%"/>

**Figure 1. -** (_ i, i-z_) diagram of USco members (left) identified by \citet{MiretRoig2022} and Taurus members (right) from \citet{Esplin2019}, represented as blue dots. The HST targets are over-plotted as red dots and the VLT targets are orange squares. The \citet{Chabrier2023} isochrones at respectively 5 and 3 Myr and 140 pc are represented by  green dashed lines and the corresponding masses are indicated on the right vertical axis. A reddening vector A$_{\rm V}$=3 mag is also represented, and the planetary mass limit of 13 $\mjup$ is indicated. The new binary candidate identified in Taurus is over-plotted as a black open circle.
 (*fig:targets_cmd*)

</div>
<div id="div_fig3">

<img src="tmp_2506.14380/./Figures/contrast_hst.png" alt="Fig4" width="100%"/>

**Figure 4. -** Detection limits in the HST F814W (top) and F850LP (bottom) images.  (*fig:contrast_hst*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.14380"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

530  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
